In [1]:
!pip install xlrd #read xls
!pip install openpyxl #read xlsx
import pandas as pd
import requests
import time

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File 

In [2]:
request_path = "http://127.0.0.1:8000/api/pesquisa/iniciacao_cientifica"

In [3]:
path = '/home/ricardo/Downloads/CPq_Dados_Atena_Mai_2022 - divulgação.xlsx'

In [4]:
listadpt = ['FLA', 'FLP', 'FLF', 'FLH', 'FLC', 'FLM', 'FLO', 'FLL', 'FSL', 'FLT', 'FLG']

listadptext = ['Antropologia', 'Ciência Política', 'Filosofia', 'História', 'Letras Clássicas e Vernáculas', 
                'Letras Modernas', 'Letras Orientais', 'Lingüística', 'Sociologia', 
                'Teoria Literária e Literatura Comparada', 'Geografia']





In [37]:
dfDiff = pd.DataFrame(columns={'codproj','COD PROJETO', 'ano_proj', 'ANO PROJETO', 'nome_discente', 
                'ALUNO', 'status_projeto', 'SITUAÇÃO PROJETO', 'titulo_pesquisa', 
                'TITULO', 'nome_departamento', 'DEPARTAMENTO'}) 

for dpt, dptext in zip(listadpt, listadptext):
    for ano in range(2016, 2021):
        parametros = {
        'ano': ano,
        'departamento': dpt,
        'departamento_ext': dptext,
        'export': 'false',
        'tipo': 'anual'
        }

        
        time.sleep(1)
        com = requests.get(request_path+"?bolsa=true", params= parametros)
        time.sleep(0.5)
        sem = requests.get(request_path+"?bolsa=false", params= parametros)
        
        try:    
            bolsa = pd.DataFrame(com.json())
            sem_bolsa = pd.DataFrame(sem.json())
        except:
            print(parametros)
            print(com.status_code) 
            print('\n')
            continue
            
        portal = pd.concat([bolsa, sem_bolsa], ignore_index=True)

        cpq = pd.read_excel(path)

        cpq = cpq.loc[(cpq['DEPARTAMENTO'] == parametros['departamento_ext']) & (cpq['ANO PROJETO'] == int(parametros['ano'])) ]
        cpq = cpq.reset_index(drop=True)

        portal = portal[['codproj', 'ano_proj', 'nome_discente', 'titulo_pesquisa', 'status_projeto', 'nome_departamento']]
        cpq =  cpq[['COD PROJETO', 'ANO PROJETO', 'ALUNO', 'TITULO', 'SITUAÇÃO PROJETO', 'DEPARTAMENTO']]

        cpq = cpq.astype({'COD PROJETO' : str, 'ANO PROJETO' : str})
        portal = portal.astype({'codproj' : str, 'ano_proj' : str})

        df = pd.merge(
        portal, 
        cpq, 
        how='outer', 
        left_on=['codproj', 'ano_proj'], 
        right_on=['COD PROJETO', 'ANO PROJETO']
                        )

        column_names = ['codproj','COD PROJETO', 'ano_proj', 'ANO PROJETO', 'nome_discente', 
            'ALUNO', 'status_projeto', 'SITUAÇÃO PROJETO', 'titulo_pesquisa', 
            'TITULO', 'nome_departamento', 'DEPARTAMENTO']

        df = df.reindex(columns=column_names)
        df = df.sort_values(by=['ALUNO'], ascending=True)
        df = df.reset_index(drop=True)
       
        #df = df.loc[df['nome_discente'].isnull() | df['ALUNO'].isnull()]
        
        dfDiff=dfDiff.append( [df] ) # deprecated
        dfDiff = pd.concat([dfDiff, df])

          

In [32]:
column_names = ['codproj','COD PROJETO', 'ano_proj', 'ANO PROJETO', 'nome_discente', 
            'ALUNO', 'status_projeto', 'SITUAÇÃO PROJETO', 'titulo_pesquisa', 
            'TITULO', 'nome_departamento', 'DEPARTAMENTO']

dfDiff = dfDiff.reindex(columns=column_names)
dfDiff = dfDiff.sort_values(by=['ALUNO'], ascending=True)
dfDiff = dfDiff.reset_index(drop=True)
dfDiff

,codproj,COD PROJETO,ano_proj,ANO PROJETO,nome_discente,ALUNO,status_projeto,SITUAÇÃO PROJETO,titulo_pesquisa,TITULO,nome_departamento,DEPARTAMENTO
0,NaN,2784,NaN,2020,NaN,Bruno Batista de Sousa,NaN,Ativo,NaN,"Compilação, tradução e análise de lemas latino...",NaN,Letras Clássicas e Vernáculas
1,NaN,514,NaN,2020,NaN,Bruno Fontana Nishiyama Bernardes Ferreira,NaN,Ativo,NaN,Platão e o Mito,NaN,Filosofia
2,NaN,3064,NaN,2020,NaN,Bruno Mariano Horemans,NaN,Ativo,NaN,História e Memória na ficção de Valêncio Xavier,NaN,Letras Clássicas e Vernáculas
3,NaN,1276 e 3910,NaN,2018,NaN,Gabriel Rodrigues Sanches Cordeiro,NaN,Transferido (2018-3909),NaN,Fome e abastecimento na região parisiense entr...,NaN,História
4,NaN,2313,NaN,2020,NaN,Gabrielly Borges,NaN,Ativo,NaN,Ou isto ou aquilo: discussão do estatuto de li...,NaN,Letras Clássicas e Vernáculas
5,NaN,306 e 998,NaN,2020,NaN,João Cândido Cartocci Maia,NaN,Aprovado,NaN,"Amar-amaro: desejo, política e poesia no prime...",NaN,Teoria Literária e Literatura Comparada
6,NaN,2693,NaN,2020,NaN,Lucas Augusto dos Santos Camargo,NaN,Ativo (2020-1022),NaN,Método automatizado para mapeamento de habitat...,NaN,Geografia
7,NaN,2671,NaN,2020,NaN,Vitor Sposito de Faria,NaN,Concluido,NaN,Cultive apenas borboletas no seu estômago. Die...,NaN,História
8,46,NaN,2016,NaN,Daniel Tamayo de Lima,NaN,Transferido,NaN,Projeto de Iniciação Científica,NaN,Filosofia,NaN
9,3910,NaN,2018,NaN,Gabriel Rodrigues Sanches Cordeiro,NaN,Aprovado,NaN,Fome e Abastecimento na Região Parisiense Entr...,NaN,História,NaN


In [6]:
dfDiff.to_csv('ics diferenças dados - cpq (2).csv', encoding='utf-8', index=False)

PASSO A PASSO ====>>

In [13]:
pd.reset_option("display.max_rows", "display.max_columns")

In [8]:
parametros = {
    'ano': "2019",
    'departamento': 'FLM',
    'departamento_ext': 'Letras Modernas',
    'export': 'false',
    'tipo': 'anual'
}

In [9]:
request_path = "http://127.0.0.1:8000/api/pesquisa/iniciacao_cientifica"

In [11]:
com = requests.get(request_path+"?bolsa=true", params= parametros)
sem = requests.get(request_path+"?bolsa=false", params= parametros)

In [12]:
bolsa = pd.DataFrame(com.json())
sem_bolsa = pd.DataFrame(sem.json())

In [13]:
portal = pd.concat([bolsa, sem_bolsa], ignore_index=True)

In [14]:
print("n bolsa: ",len(bolsa), "n sem bolsa: ", len(sem_bolsa), ' | Total: ', len(bolsa) + len(sem_bolsa))

n bolsa:  1 n sem bolsa:  24  | Total:  25


In [15]:
path = '/home/ricardo/Downloads/CPq_Dados_Atena_Mai_2022 - divulgação.xlsx'

In [16]:
cpq = pd.read_excel(path)

In [17]:
cpq.DEPARTAMENTO.unique()

array(['Antropologia', 'Ciência Política', 'Filosofia', 'Geografia',
       'História', 'Letras Clássicas e Vernáculas', 'Letras Modernas',
       'Letras Orientais', 'Lingüística', 'Sociologia',
       'Teoria Literária e Literatura Comparada'], dtype=object)

In [18]:
cpq.loc[(cpq['DEPARTAMENTO'] == parametros['departamento_ext'])]

,ANO PROJETO,COD PROJETO,UNIDADE,DEPARTAMENTO,DATA INICIO PROJETO,MESES DURAÇÃO,DATA FIM PROJETO,CODIGO AREA CONHECIMENTO,AREA CONHECIMENTO,PALAVRA CHAVE,...,AREA PRIORITARIA MCTIC,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
109,2016,2404,FFLCH,Letras Modernas,2016-09-01,12.133333,31/08/2017,80106005,"Lingüística, Letras e Artes-Lingüística Aplicada",língua alemã; linguística; sintaxe do alemão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,2016,278,FFLCH,Letras Modernas,2016-11-01,12.133333,31/10/2017,80208002,"Lingüística, Letras e Artes-Literaturas Estran...",conto; literatura; história,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,2016,3592,FFLCH,Letras Modernas,2017-02-01,12.133333,31/01/2018,80202004,"Lingüística, Letras e Artes-Línguas Estrangeir...",língua italiana;material didático;cultura ital...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,2016,2002,FFLCH,Letras Modernas,2016-11-01,12.133333,31/10/2017,80202004,"Lingüística, Letras e Artes-Línguas Estrangeir...",ensino de espanhol; currículo; Ensino Médio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,2016,3572,FFLCH,Letras Modernas,2016-12-01,12.133333,30/11/2017,80202004,"Lingüística, Letras e Artes-Línguas Estrangeir...",Breakfast at Tifany's;Audrey Hepburn;Truman Ca...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,2021,335,FFLCH,Letras Modernas,2021-04-01,12.133333,31/03/2022,80208002,"Lingüística, Letras e Artes-Literaturas Estran...",adaptação; obra literária occitana; album musical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1493,2022,314,FFLCH,Letras Modernas,2022-03-14,12.133333,13/03/2023,80205003,"Lingüística, Letras e Artes-Teoria Literária",Georg Lukács; Mikhail Bakhtin; teoria do romance,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1494,2022,233,FFLCH,Letras Modernas,2021-11-27,12.133333,26/11/2022,80202004,"Lingüística, Letras e Artes-Línguas Estrangeir...",língua alemã; livro didático; vocabulário,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1495,2022,165,FFLCH,Letras Modernas,2021-12-01,12.133333,30/11/2022,80208002,"Lingüística, Letras e Artes-Literaturas Estran...",poesia chilena; Soledad Fariña; poesia contemp...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
cpq = cpq.loc[(cpq['DEPARTAMENTO'] == parametros['departamento_ext']) & (cpq['ANO PROJETO'] == int(parametros['ano'])) ]
cpq = cpq.reset_index(drop=True)

In [20]:
print("Total Portal Dados: ", len(portal), '. Total CPQ: ', len(cpq))

Total Portal Dados:  25 . Total CPQ:  25


In [21]:
portal = portal[['codproj', 'ano_proj', 'nome_discente', 'titulo_pesquisa', 'status_projeto', 'nome_departamento']]
cpq =  cpq[['COD PROJETO', 'ANO PROJETO', 'ALUNO', 'TITULO', 'SITUAÇÃO PROJETO', 'DEPARTAMENTO']]

In [22]:
cpq.dtypes

COD PROJETO         object
ANO PROJETO          int64
ALUNO               object
TITULO              object
SITUAÇÃO PROJETO    object
DEPARTAMENTO        object
dtype: object

In [23]:
cpq = cpq.astype({'COD PROJETO' : str, 'ANO PROJETO' : str})


In [24]:
portal.dtypes

codproj               int64
ano_proj              int64
nome_discente        object
titulo_pesquisa      object
status_projeto       object
nome_departamento    object
dtype: object

In [25]:
portal = portal.astype({'codproj' : str, 'ano_proj' : str})


In [26]:
portal.dtypes

codproj              object
ano_proj             object
nome_discente        object
titulo_pesquisa      object
status_projeto       object
nome_departamento    object
dtype: object

In [27]:
df = pd.merge(
    portal, 
    cpq, 
    how='outer', 
    left_on=['codproj', 'ano_proj'], 
    right_on=['COD PROJETO', 'ANO PROJETO']
)

In [28]:
column_names = ['codproj','COD PROJETO', 'ano_proj', 'ANO PROJETO', 'nome_discente', 
                'ALUNO', 'status_projeto', 'SITUAÇÃO PROJETO', 'titulo_pesquisa', 
                'TITULO', 'nome_departamento', 'DEPARTAMENTO']

df = df.reindex(columns=column_names)
df = df.sort_values(by=['ALUNO'], ascending=True)
df = df.reset_index(drop=True)

In [29]:
print("Total Portal Dados: ", len(portal), '. Total CPQ: ', len(cpq))

Total Portal Dados:  25 . Total CPQ:  25


In [30]:
df

,codproj,COD PROJETO,ano_proj,ANO PROJETO,nome_discente,ALUNO,status_projeto,SITUAÇÃO PROJETO,titulo_pesquisa,TITULO,nome_departamento,DEPARTAMENTO
0,143,143,2019,2019,Ana Carolina de Assis Perfeito,Ana Carolina de Assis Perfeito,Aprovado,Aprovado,Aspectos interculturais para preparação à mobi...,Aspectos interculturais para preparação à mobi...,Letras Modernas,Letras Modernas
1,3056,3056,2019,2019,Ana Clara Neves Silveira,Ana Clara Neves Silveira,Aprovado,Aprovado,Licenciatura em língua alemã: processos de con...,Licenciatura em língua alemã: processos de con...,Letras Modernas,Letras Modernas
2,2122,2122,2019,2019,Ayne Silva Soares,Ayne Silva Soares,Aprovado,Aprovado,O gênero Diário de Leitura como instrumento pa...,O gênero Diário de Leitura como instrumento pa...,Letras Modernas,Letras Modernas
3,468,468,2019,2019,Beatriz de Souza Santos,Beatriz de Souza Santos,Aprovado,Aprovado,Windrose der Qualen (Rosa dos ventos dos lamen...,Windrose der Qualen (Rosa dos ventos dos lamen...,Letras Modernas,Letras Modernas
4,3730,3730,2019,2019,Camila Marcucci Schmidt,Camila Marcucci Schmidt,Aprovado,Aprovado,A transferência linguística na aquisição da po...,A transferência linguística na aquisição da po...,Letras Modernas,Letras Modernas
5,318,318,2019,2019,Carolina Martins Pedroso,Carolina Martins Pedroso,Aprovado,Aprovado,Sprachlernberatung: a consultoria voltada para...,Sprachlernberatung: a consultoria voltada para...,Letras Modernas,Letras Modernas
6,2234,2234,2019,2019,Cesar Henrique Fachina,Cesar Henrique Fachina,Aprovado,Aprovado,GÊNESE E DIGÊNESE:\nUMA RELAÇÃO DE PODER NA FU...,GÊNESE E DIGÊNESE:\nUMA RELAÇÃO DE PODER NA FU...,Letras Modernas,Letras Modernas
7,3416,3416,2019,2019,Dayane Manfrere Alves,Dayane Manfrere Alves,Aprovado,Aprovado,O processo de tradução comentada de Bestie de ...,O processo de tradução comentada de Bestie de ...,Letras Modernas,Letras Modernas
8,411,411,2019,2019,Douglas Patriota Ferreira,Douglas Patriota Ferreira,Aprovado,Aprovado,A parada glótica no ensino de alemão como líng...,A parada glótica no ensino de alemão como líng...,Letras Modernas,Letras Modernas
9,2960,2960,2019,2019,Gabriela Hikari Tomizuka,Gabriela Hikari Tomizuka,Aprovado,Aprovado,QUESTÕES DE REPRESENTAÇÃO NO CINEMA: OS ESTERE...,QUESTÕES DE REPRESENTAÇÃO NO CINEMA: OS ESTERE...,Letras Modernas,Letras Modernas


In [57]:
df.loc[df['nome_discente'].isnull() | df['ALUNO'].isnull()]

,codproj,COD PROJETO,ano_proj,ANO PROJETO,nome_discente,ALUNO,status_projeto,SITUAÇÃO PROJETO,titulo_pesquisa,TITULO,nome_departamento,DEPARTAMENTO


In [58]:
df['nome_discente'].duplicated().any() # EXISTE ALUNO REPETIDO?

False

In [115]:
df.loc[df['nome_discente'].duplicated()] # & df['nome_discente'] is not None

,codproj,COD PROJETO,ano_proj,ANO PROJETO,nome_discente,ALUNO,status_projeto,SITUAÇÃO PROJETO,titulo_pesquisa,TITULO,DEPARTAMENTO


In [116]:
obs = '''\n\nHistória 2021: 
            3336, 2821,3620 -> Projeto na planilha cpq ativo, mas agr no replicadoe stá cancelado (Projeto cancelado segundo solicitação do orientador.)
            962 -> Status transferido, substituição bolsista, vigencia > 30 dias
           Em geral está tudo certo: poral 32, cpq 33'''

In [117]:
nome_arquivo = "observações sobre ic (equalizando dados com o cpq)"
try:
    arquivo = open(nome_arquivo, 'r+')#edita arquivo
except FileNotFoundError:
    arquivo = open(nome_arquivo, 'w+')#cria o arquivo
    
arquivo.readlines()
arquivo.writelines(obs)
arquivo.close()





História 2018: Há dois projetos com status transferido e que consta na planilha da cpq 
        (1276 - Gabriel Sanches, 1165 - Rafael Gonçalves)
         O 1276 ele é transferido pois a bolsa mudou, de PIBIC foi para FAPESP.
         O 1165 foi transferido pq mudou o bolsista, mas deve contar pq a vigencia é de 2 meses.
         Mas no geral Portal: 40 CPQ: 41 (O CPQ junta os projetos do gabeial 1276 e 3910), a diferença é do Raphael que não consta no portal, pois o status é transferido
         PROJ COM VIGENCIA MENOR QUE 30 DIAS DEVE SER REMOVIDO
História 2020: Tem um projeto cancelado recentemente no replicado, mas que na planilha da cris aparecia como concluído. Como agora está cancelado então vamos desconsiderar.
           Em geral está tudo certo: poral 32, cpq 33
História 2021: 
            3336, 2821, -> Projeto na planilha cpq ativo, mas agr no replicadoe stá cancelado (Projeto cancelado segundo solicitação do orientador.)
            962 -> Status transferido, substituição bolsista, vigencia > 30 dias
           Em geral está tudo certo: poral 32, cpq 33


In [27]:
%pwd


'/home/ricardo'

In [332]:
#https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=166088 sha256=60b5f47f73b306d6dce254dfd605ef9b1f8cecda84698042e154974a38f12ed7
  Stored in directory: /home/ricardo/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein


In [333]:
import Levenshtein

In [343]:
Levenshtein.distance("Fome e abastecimento na região parisiense entre os séculos VII e XI", 'Fome e Abastecimento na Região Parisiense Entre os Séculos VII e XI')

5

In [344]:
Levenshtein.distance("Amar-amaro: desejo, política e poesia no primeiro Drummond.", "Amar-amaro: desejo, política e poesia no primeiro Drummond")

1